In [ ]:
#Python Code to convert into TFLite model
import tensorflow as tf
from keras.saving import load_model  


saved_model = "LSTM_model.h5"  
model = load_model(saved_model)


converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  
    tf.lite.OpsSet.SELECT_TF_OPS  
]


tflite_model = converter.convert()


tflite_converted_model = "LSTM_model.tflite"
with open(tflite_converted_model, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model successfully converted and saved to {tflite_conerted_model}")


In [ ]:
#Python Code to load TFLite model and and get prediction 
from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf

app = Flask(__name__)

interpreter = tf.lite.Interpreter(model="LSTM_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

class_labels = ["No CTS", "Mild CTS", "Moderate CTS", "Severe CTS"]

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        emg_values = data.get("emg", [])

        if len(emg_values) != 10:
            return jsonify({"error": "10 EMG values required"}), 400

        input_data = np.array(emg_values, dtype=np.float32).reshape(-1, 10, 1)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        output = interpreter.get_tensor(output_details[0]['index'])
        prediction_index = int(np.argmax(output))
        prediction_label = class_labels[prediction_index]

        base_angle = int(emg_values[-1] * 100)
        base_angle = max(0, min(base_angle, 90))

        return jsonify({
            "prediction": prediction_label,
            "label": prediction_index,
            "angle": base_angle,
            "confidence": output[0].tolist()
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)